# Xây dựng **Search Database** từ **Span Adaptation Vectors (TRAIN)**

- **Mục đích**
  - Gom toàn bộ **Span AV** của tập **TRAIN** (từ cả GramVar & ParaVE) thành một **cơ sở dữ liệu** để truy vấn/so khớp.
  - Chuẩn hoá cấu trúc dữ liệu: xếp **N lớp đầu** (mặc định `12`) theo thứ tự và lưu kèm **metadata**.

- **Đầu vào**
  - Thư mục Span AV (TRAIN):
    ```
    .../span_adaptation_vectors_train_gramvar/
    .../span_adaptation_vectors_train_parave/
    ```
  - Mỗi file:  
    `.../<verb_name>/sentence_<i>_span_av.pt`  
    là **dict**: `{ "ARG0": { "layer_0": Tensor[2304], ... }, ... }`
    > Ở đây script đang đọc trực tiếp **theo lớp**: `layer_1 … layer_12`.  
    Bạn có thể đổi số lớp qua biến `NUM_LAYERS_TO_MATCH`.

- **Đầu ra (cache)**
  - Thư mục: `.../cached_databases/`
    - `combined_train_db_vectors.pt`  
      > Tensor **[N_span, NUM_LAYERS_TO_MATCH, 2304]**
    - `combined_train_db_metadata.json`  
      > List `{ verb_name, arg_label, original_file, source_dataset }`


- **Quy trình**
  1. **Quét** đệ quy tất cả các file `.pt` trong từng thư mục TRAIN nguồn.
  2. **Tải** dict các lớp, kiểm tra đủ `layer_1` → `layer_NUM_LAYERS_TO_MATCH`.
  3. **Ghép** theo thứ tự lớp → một tensor **[NUM_LAYERS_TO_MATCH, 2304]** cho mỗi span; **append** vào DB.
  4. **Ghi metadata** cho từng span: tên động từ, nhãn ARG, file gốc, nguồn (GramVar/ParaVE).
  5. **Lưu cache**: vectors `.pt` + metadata `.json`.

In [ ]:
# Import các thư viện cần thiết
import torch
import os
import json
import glob
from tqdm.auto import tqdm

# CẤU HÌNH
NUM_LAYERS_TO_MATCH = 12

# --- THIẾT LẬP CÁC ĐƯỜNG DẪN ---
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'

# Danh sách các thư mục chứa dữ liệu TRAIN để xây dựng DB
train_span_av_dirs = [
    os.path.join(drive_base_path, 'Span Adaptation Vector/Without Weight/Train_Content/span_adaptation_vectors_train_gramvar'),
    os.path.join(drive_base_path, 'Span Adaptation Vector/Without Weight/Train_Content/span_adaptation_vectors_train_parave')
]

# Đường dẫn để lưu database đã xử lý
db_cache_dir = os.path.join(drive_base_path, 'cached_databases')
os.makedirs(db_cache_dir, exist_ok=True)
db_vectors_file = os.path.join(db_cache_dir, 'combined_train_db_vectors.pt')
db_metadata_file = os.path.join(db_cache_dir, 'combined_train_db_metadata.json')

# --- 2. HÀM XÂY DỰNG DATABASE ---

def build_search_database(span_av_dirs, num_layers):
    """
    Tải TẤT CẢ CÁC LỚP của span vector bằng cách quét NHIỀU thư mục.
    """
    print(f"\n--- Bắt đầu xây dựng cơ sở dữ liệu tìm kiếm từ {len(span_av_dirs)} thư mục ---")
    database_vectors = []
    database_metadata = []

    for span_av_dir in span_av_dirs:
        print(f"  Đang quét thư mục: {os.path.basename(span_av_dir)}")
        all_files = glob.glob(os.path.join(span_av_dir, '**', '*.pt'), recursive=True)

        for file_path in tqdm(all_files, desc=f"Xây dựng DB từ {os.path.basename(span_av_dir)}"):
            try:
                filename_parts = os.path.basename(file_path).replace('.pt', '').split('_')
                arg_label = f"{filename_parts[-2]}-{filename_parts[-1]}"
                verb_name = os.path.basename(os.path.dirname(file_path))

                layers_data = torch.load(file_path, map_location='cpu')
                if all(f'layer_{j+1}' in layers_data for j in range(num_layers)):
                    layer_vectors = [layers_data[f'layer_{j+1}'] for j in range(num_layers)]
                    database_vectors.append(torch.stack(layer_vectors))
                    database_metadata.append({
                        'verb_name': verb_name,
                        'arg_label': arg_label,
                        'original_file': os.path.basename(file_path),
                        'source_dataset': os.path.basename(span_av_dir)
                    })
            except Exception:
                continue

    if not database_vectors:
        return None, None

    return torch.stack(database_vectors), database_metadata

# --- 3. THỰC THI CHƯƠNG TRÌNH ---
if __name__ == "__main__":
    if os.path.exists(db_vectors_file) and os.path.exists(db_metadata_file):
        print("Cơ sở dữ liệu Train đã tồn tại. Không cần xây dựng lại.")
    else:
        train_db_vectors, train_db_metadata = build_search_database(train_span_av_dirs, NUM_LAYERS_TO_MATCH)
        if train_db_vectors is not None:
            print(f" -> Xây dựng thành công DB với {len(train_db_metadata)} span.")
            print(f"Đang lưu database vào '{db_cache_dir}'...")
            torch.save(train_db_vectors, db_vectors_file)
            with open(db_metadata_file, 'w', encoding='utf-8') as f:
                json.dump(train_db_metadata, f, ensure_ascii=False)
            print(" -> Lưu database thành công.")
        else:
            print("\nLỖI: Không thể xây dựng cơ sở dữ liệu train.")

    print("\n--- QUÁ TRÌNH XÂY DỰNG DATABASE ĐÃ KẾT THÚC ---")


--- Bắt đầu xây dựng cơ sở dữ liệu tìm kiếm từ 2 thư mục ---
  Đang quét thư mục: span_adaptation_vectors_train_gramvar


Xây dựng DB từ span_adaptation_vectors_train_gramvar:   0%|          | 0/10390 [00:00<?, ?it/s]

  Đang quét thư mục: span_adaptation_vectors_train_parave


Xây dựng DB từ span_adaptation_vectors_train_parave:   0%|          | 0/5131 [00:00<?, ?it/s]

 -> Xây dựng thành công DB với 15521 span.
Đang lưu database vào '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/cached_databases'...
 -> Lưu database thành công.

--- QUÁ TRÌNH XÂY DỰNG DATABASE ĐÃ KẾT THÚC ---


- **Cấu trúc thư mục (minh hoạ)**
  ```bash
  .../span_adaptation_vectors_train_gramvar/
  ├── bind/
  │   ├── sentence_0_span_av.pt
  │   └── sentence_1_span_av.pt
  .../span_adaptation_vectors_train_parave/
  ├── regulate/
  │   ├── sentence_3_span_av.pt
  │   └── ...
  .../cached_databases/
  ├── combined_train_db_vectors.pt
  └── combined_train_db_metadata.json